In [1]:
from keras.utils.np_utils import to_categorical
from keras.models import Model
from keras.layers import Input, Dense, Dropout
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import math
import time
import itertools

Using TensorFlow backend.


In [2]:
#Read first dataset
dataset1 = pd.read_csv('./mushrooms.csv')
#np.count_nonzero(dataset1['Outcome'])/len(dataset1['Outcome'])

In [3]:
continuous=['class']
dataset1_dummy = pd.DataFrame()
for col in dataset1.columns:
    if col not in continuous:
        uniques = np.unique(dataset1[col])
        for unique in uniques:
            name = str(col)+'_'+str(unique)
            dataset1_dummy[name]=(dataset1[col]==unique)
    elif col=='class':
        dataset1_dummy['class1']=(dataset1[col]=='p')
    else:
        dataset1_dummy[col]=dataset1[col]
(dataset1_dummy)

,class1,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_f,cap-surface_g,cap-surface_s,...,population_s,population_v,population_y,habitat_d,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,True,False,False,False,False,False,True,False,False,True,...,True,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,True,False,False,True,...,False,False,False,False,True,False,False,False,False,False
2,False,True,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,True,False,False,False
3,True,False,False,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,False,True,False
4,False,False,False,False,False,False,True,False,False,True,...,False,False,False,False,True,False,False,False,False,False
5,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,True,False,False,False,False,False
6,False,True,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,True,False,False,False
7,False,True,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,True,False,False,False
8,True,False,False,False,False,False,True,False,False,False,...,False,True,False,False,True,False,False,False,False,False
9,False,True,False,False,False,False,False,False,False,True,...,True,False,False,False,False,False,True,False,False,False


In [4]:
'''print(len(dataset1_dummy))
indexes = dataset1_dummy[dataset1_dummy['class1']==1].index
idp = dataset1_dummy[dataset1_dummy['class1']==0].iloc[0:len(indexes)].index
indexes = indexes.append(idp)
dataset1 = dataset1_dummy.iloc[indexes]
dataset1 = dataset1_dummy.sort_index()
np.count_nonzero(dataset1_dummy['class1'])/len(dataset1_dummy['class1'])'''

"print(len(dataset1_dummy))\nindexes = dataset1_dummy[dataset1_dummy['class1']==1].index\nidp = dataset1_dummy[dataset1_dummy['class1']==0].iloc[0:len(indexes)].index\nindexes = indexes.append(idp)\ndataset1 = dataset1_dummy.iloc[indexes]\ndataset1 = dataset1_dummy.sort_index()\nnp.count_nonzero(dataset1_dummy['class1'])/len(dataset1_dummy['class1'])"

In [5]:
#Read Second dataset
dataset2 = pd.read_csv('adult.csv')
dataset2.head(5)

,age,workclass,fnlwgt,education,education_num,marital_state,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_county,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [6]:
continuous = ['age','fnlwgt','capital_gain','capital_loss','hours_per_week','class']
dataset2

,age,workclass,fnlwgt,education,education_num,marital_state,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_county,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


In [7]:
dataset2_dummy = pd.DataFrame()
for col in dataset2.columns:
    if col not in continuous:
        uniques = np.unique(dataset2[col])
        for unique in uniques:
            name = str(col)+'_'+str(unique)
            dataset2_dummy[name]=(dataset2[col]==unique)
    elif col=='class':
        dataset2_dummy['class2']=(dataset2[col]==' <=50K')
    else:
        dataset2_dummy[col]=dataset2[col]
(dataset2_dummy)

,age,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,workclass_ Without-pay,...,native_county_ Puerto-Rico,native_county_ Scotland,native_county_ South,native_county_ Taiwan,native_county_ Thailand,native_county_ Trinadad&Tobago,native_county_ United-States,native_county_ Vietnam,native_county_ Yugoslavia,class2
0,39,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,True,False,False,True
1,50,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,True,False,False,True
2,38,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,True
3,53,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,True
4,28,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
5,37,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,True
6,49,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
7,52,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
8,31,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
9,42,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False


In [8]:
indexes = dataset2_dummy[(dataset2_dummy['class2'])==0].index
idp = dataset2_dummy[(dataset2_dummy['class2'])==1].iloc[0:len(indexes)].index
indexes=indexes.append(idp)
dataset2_dummy = dataset2_dummy.iloc[indexes]
dataset2_dummy


,age,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,workclass_ Without-pay,...,native_county_ Puerto-Rico,native_county_ Scotland,native_county_ South,native_county_ Taiwan,native_county_ Thailand,native_county_ Trinadad&Tobago,native_county_ United-States,native_county_ Vietnam,native_county_ Yugoslavia,class2
7,52,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
8,31,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
9,42,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
10,37,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
11,30,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
14,40,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
19,43,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,True,False,False,False
20,40,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
25,56,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
27,54,True,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False


In [9]:
dataset1_back = dataset1
dataset1 = dataset1_dummy.sort_index()
dataset2_back = dataset2
dataset2 = dataset2_dummy.sort_index()
#np.count_nonzero(dataset2['class'])/len(dataset2['class'])

In [10]:
dataset_new = pd.DataFrame(columns=list(dataset1.columns)+list(dataset2.columns))
for column in dataset2.columns:
    dataset_new[column] = dataset2[column]

for column in dataset1.columns:
    dataset_new[column] = dataset1[column]




'''dataset_new = pd.DataFrame(columns=list(dataset1.columns)+list(dataset2.columns))
for column in dataset2.columns:
    dataset_new[column] = dataset2[column]
dataset_new.index = range(len(dataset_new))
indnan = 0
while indnan in dataset_new.index:
    max = indnan+len(dataset1)
    if max in dataset_new.index: 
        #print(indnan)
        for column in dataset1.columns:
            dataset_new[column][indnan:indnan+len(dataset1)] = dataset1[column]
    indnan+=len(dataset1[column])

dataset_new'''
dataset_new

,class1,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_f,cap-surface_g,cap-surface_s,...,native_county_ Puerto-Rico,native_county_ Scotland,native_county_ South,native_county_ Taiwan,native_county_ Thailand,native_county_ Trinadad&Tobago,native_county_ United-States,native_county_ Vietnam,native_county_ Yugoslavia,class2
0,True,False,False,False,False,False,True,False,False,True,...,False,False,False,False,False,False,True,False,False,True
1,False,False,False,False,False,False,True,False,False,True,...,False,False,False,False,False,False,True,False,False,True
2,False,True,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,True,False,False,True
3,True,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,True,False,False,True
4,False,False,False,False,False,False,True,False,False,True,...,False,False,False,False,False,False,False,False,False,True
5,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,True,False,False,True
6,False,True,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,True
7,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
8,True,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,True,False,False,False
9,False,True,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,True,False,False,False


In [11]:
'''indexes_null = (dataset_new['class1'].isnull())
dataset_new['class1'][indexes_null]=(np.random.randint(0,2,len(dataset_new['class1'][indexes_null]) ))

np.count_nonzero(dataset_new['class1'])/len(dataset_new['class1'])'''

"indexes_null = (dataset_new['class1'].isnull())\ndataset_new['class1'][indexes_null]=(np.random.randint(0,2,len(dataset_new['class1'][indexes_null]) ))\n\nnp.count_nonzero(dataset_new['class1'])/len(dataset_new['class1'])"

In [12]:
dataset_new=dataset_new.fillna(0)
y = [dataset_new['class1'],dataset_new['class2']]
dataset_new = dataset_new.drop('class1',axis=1)
dataset_new = dataset_new.drop('class2',axis=1)
len(dataset_new)

15682

In [13]:
X = np.asarray(dataset_new)
X_train, X_test, Y0_train, Y0_test ,Y1_train, Y1_test  = train_test_split( X, y[0],y[1], test_size=0.30, random_state=42)




In [14]:

Y0=to_categorical(Y0_train,num_classes=2)
Y1=to_categorical(Y1_train,num_classes=2)
Y0_test=to_categorical(Y0_test,num_classes=2)
Y1_test=to_categorical(Y1_test,num_classes=2)


In [136]:
a = Input(shape=(len(dataset_new.columns),))
x = Dense(8, activation='sigmoid')(a) # this is your network, let's say you have 2 hidden layers of 64 nodes each (don't know if that's enough for you)
x = Dropout(0.5)(x)
x = Dense(8, activation='sigmoid')(x)
x = Dense(8, activation='sigmoid')(x)
x = Dense(8, activation='sigmoid')(x)
x = Dense(8, activation='sigmoid')(x)
x = Dense(8, activation='sigmoid')(x)
x = Dense(8, activation='sigmoid')(x)
x = Dense(8, activation='sigmoid')(x)
x = Dense(8, activation='sigmoid')(x)
x = Dense(8, activation='sigmoid')(x)
x = Dense(8, activation='sigmoid')(x)
x = Dropout(0.5)(x)
#x = Dense(64, activation='relu')(x)
b1 = Dense(2,activation='sigmoid')(x)
b2 = Dense(2,activation='sigmoid')(x)

model = Model(inputs=[a], outputs=[b1, b2])
model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy'], optimizer='adam', metrics=['accuracy'])
#model = Model(inputs=[a], outputs=[b1])
#model.compile(loss=['categorical_crossentropy'], optimizer='adam', metrics=['accuracy'])



In [137]:
model.fit(X_train, [Y0,Y1], epochs=15, batch_size=1, verbose=2)
#model.fit(X_train, [Y0,Y1], epochs=10, batch_size=1, verbose=2)

#model.fit(X, [Y0], epochs=100, batch_size=1, verbose=2)

Epoch 1/15
14s - loss: 1.2717 - dense_209_loss: 0.5724 - dense_210_loss: 0.6993 - dense_209_acc: 0.7472 - dense_210_acc: 0.4974
Epoch 2/15
14s - loss: 1.2552 - dense_209_loss: 0.5616 - dense_210_loss: 0.6936 - dense_209_acc: 0.7529 - dense_210_acc: 0.4982
Epoch 3/15
14s - loss: 1.2539 - dense_209_loss: 0.5603 - dense_210_loss: 0.6935 - dense_209_acc: 0.7529 - dense_210_acc: 0.4960
Epoch 4/15
14s - loss: 1.2529 - dense_209_loss: 0.5595 - dense_210_loss: 0.6934 - dense_209_acc: 0.7529 - dense_210_acc: 0.4981
Epoch 5/15
14s - loss: 1.2526 - dense_209_loss: 0.5592 - dense_210_loss: 0.6934 - dense_209_acc: 0.7529 - dense_210_acc: 0.4929
Epoch 6/15
14s - loss: 1.2526 - dense_209_loss: 0.5593 - dense_210_loss: 0.6934 - dense_209_acc: 0.7529 - dense_210_acc: 0.4939
Epoch 7/15
14s - loss: 1.2528 - dense_209_loss: 0.5594 - dense_210_loss: 0.6934 - dense_209_acc: 0.7529 - dense_210_acc: 0.4940
Epoch 8/15
14s - loss: 1.2526 - dense_209_loss: 0.5593 - dense_210_loss: 0.6933 - dense_209_acc: 0.7529 

In [138]:
model.evaluate(X_test,[Y0_test,Y1_test])


4416/4705 [===========================>..] - ETA: 0s 

[1.2620130789774004,
 0.56889805343401922,
 0.69311502311738971,
 0.74410201912858664,
 0.50626992561105211]

In [139]:
#np.count_nonzero(np.round(predictions[0][:,0]))/len(predictions[0][:,0]),np.count_nonzero(np.round(Y0_test[:,0]))/len(Y0_test[:,0])

In [140]:
dict_right={}
dict_right[0]=[]
dict_right[1]=[]
num0 = 0
num1 = 0
predictions  = model.predict(X_test)
for row in range(len(predictions[0])):
    if np.argmax(predictions[0][row])==np.argmax(Y0_test[row]):
        dict_right[np.argmax(Y0_test[row])].append(1)
    if np.argmax(Y0_test[row])==0:
        num0+=1
    else:
        num1+=1
print(len(dict_right[0])/num0  )
print(len(dict_right[1])/num1  )  

1.0
0.0


In [141]:
dict_right={}
dict_right[0]=[]
dict_right[1]=[]
num0 = 0
num1 = 0
predictions  = model.predict(X_test)
for row in range(len(predictions[1])):
    if np.argmax(predictions[1][row])==np.argmax(Y1_test[row]):
        dict_right[np.argmax(Y1_test[row])].append(1)
    if np.argmax(Y1_test[row])==0:
        num0+=1
    else:
        num1+=1
print(len(dict_right[0])/num0  )
print(len(dict_right[1])/num1  )   

0.0
1.0
